In [1]:
from NLPUtils.preprocessUtils import removePunctuation, removeWord
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import pickle

In [2]:
MAXLEN = 10

In [3]:
trainDataFile = './SLTCDataset/mr/text_train.txt'
trainLabelFile = './SLTCDataset/mr/label_train.txt'
xTrianSavePath = './SLTCDataset/mr/x_train.npy'
yTrianSavePath = './SLTCDataset/mr/y_train.npy'

testDataFile = './SLTCDataset/mr/text_test.txt'
testLabelFile = './SLTCDataset/mr/label_test.txt'
xTestSavePath = './SLTCDataset/mr/x_test.npy'
yTestSavePath = './SLTCDataset/mr/y_test.npy'

In [4]:
removeWordList = stopwords.words('english')

In [5]:
trainRawDatas = []
voc = set()

In [6]:
with open(trainDataFile, 'r', encoding='utf-8') as f:
    for data in f:
        # remove  \n
        data = data.replace('\n', '')
        # removePunctuation
        data = removePunctuation(data)
        # toList
        data = data.split()
        # removeStopWord
        data = removeWord(removeWordList=removeWordList, sentence = data)
        
        trainRawDatas.append(data)
        voc.update(data)

In [7]:
with open('./SLTCDataset/mr/voc.data', 'wb') as f:
    pickle.dump(voc, f)

In [8]:
print(f'Len of voc {len(voc)}')

Len of voc 16610


In [9]:
word2idx = {word : i for i , word in enumerate(voc, start=1)}
word2idx.update({'<unk>': 0})

In [10]:
trainDatas = []
for data in trainRawDatas:
    trainData = [word2idx[word] for word in data[:MAXLEN]]
    while len(trainData) < MAXLEN:
        trainData.append(0)
    trainDatas.append(trainData)
x_train = np.array(trainDatas, dtype=np.int64)
np.save(xTrianSavePath, x_train)
df = pd.read_csv(trainLabelFile, encoding='utf-8', header=None)
y_train = df[0].to_numpy(dtype=np.int64)
np.save(yTrianSavePath, y_train)

In [11]:
testRawDatas = []
with open(testDataFile, 'r', encoding='utf-8') as f:
    for data in f:
        # remove  \n
        data = data.replace('\n', '')
        # removePunctuation
        data = removePunctuation(data)
        # toList
        data = data.split()
        # removeStopWord
        data = removeWord(removeWordList=removeWordList, sentence = data)
        
        testRawDatas.append(data)

In [12]:
testDatas = []
for data in testRawDatas:
    testData = []
    for word in data[:MAXLEN]:
        if word in voc:
            testData.append(word2idx[word])
        else:
            testData.append(0)
    while len(testData) < MAXLEN:
        testData.append(0)
    testDatas.append(testData)
x_test = np.array(testDatas, dtype=np.int64)
np.save(xTestSavePath, x_test)
df = pd.read_csv(testLabelFile, encoding='utf-8', header=None)
y_test = df[0].to_numpy(dtype=np.int64)
np.save(yTestSavePath, y_test)